# LFSR

In [18]:
# State und maximale Periodenlänge
state = 0b1000010010110001000011
length = 4194303

# Funktion des LFSRs
def lfsr(length, state):
    outputs = []
    for i in range(length):
        outputs.append(state & 1)
        newbit = ((state ^ (state >> 1)) & 1)
        state = (state >> 1) | (newbit << 21)
    return outputs


stream = lfsr(length, state)

Number of Zeroes: 2097151
Number of Ones: 2097152


'number = 0\n\nfor bit in stream:\n    number = number << 1\n    number = number | bit'

Alle der folgenden Tests stammen aus der NIST Statistical Test Suite for Random and Pseudorandom Number Generators for Cryptographic
Applications

# Frequency (Monobit) Test

In [7]:
import math

def frequencyMonobit(stream, start, end):
    sn = 0

    for bit in stream[start:end]:
        if bit == 1:
            sn += 1
        else:
            sn -= 1

    print(f"sn: {sn}")
    sObs = math.sqrt(sn**2)/math.sqrt(len(stream))
    print(f"sObs: {sObs}")
    pValue = math.erfc(sObs/math.sqrt(2))
    print(f"p-Wert: {pValue}")

    if pValue >= 0.01:
        print("Die Sequenz hat den 'Frequency (Monobit) Test' bestanden")
    else:
        print("Die Sequenz hat den 'Frequency (Monobit) Test' bestanden")

frequencyMonobit(stream, 0, len(stream))

sn: 1
sObs: 0.0004882813082076713
p-Wert: 0.9996104078983334
Die Sequenz hat den 'Frequency (Monobit) Test' bestanden


# Frequency Test within a Block

In [8]:
import scipy

def frequencyBlock(stream, block_size):
    n = len(stream) // block_size
    start = 0
    end = block_size
    ps = 0.0
    for i in range(n):
        block = stream[start:end]
        ones_count = 0
        for bit in block:
            if bit == 1:
                ones_count += 1
        pi = ones_count / block_size
        ps += pow(pi - 0.5, 2.0)
        start += block_size
        end += block_size
    chi_squared = 4.0 * block_size * ps
    pValue = scipy.special.gammaincc(n / 2, chi_squared / 2)
    print(f"p-Wert: {pValue}")
    
    if pValue >= 0.01:
        print("Die Sequenz hat den 'Frequency Test within a Block' bestanden")
    else:
        print("Die Sequenz hat den 'Frequency Test within a Block' nicht bestanden")

frequencyBlock(stream, 41944)


p-Wert: 0.32148661901985415
Die Sequenz hat den 'Frequency Test within a Block' bestanden


# Runs Test

In [14]:
import math

def runsTest(stream):
    number = 0
    for bit in stream:
        number += bit
    pi = number / len(stream)
    if (abs(pi - 1/2)) <= (2/math.sqrt(len(stream))):
        vnObs = 0
        for i in range(len(stream) - 1):
            if not stream[i] == stream[i + 1]:
                vnObs += 1
        vnObs += 1
        pValue = math.erfc((vnObs - (2*len(stream)*pi*(1 - pi)))/(2*math.sqrt(2*len(stream))*pi*(1 - pi)))
        print(f"vnObs: {vnObs}")
        print(f"p-Wert: {pValue}")
        if pValue >= 0.01:
            print("Die Sequenz hat den 'Runs Test' bestanden")
        else:
            print("Die Sequenz hat den 'Runs Test' nicht bestanden")

    else:
        print("Der Test wurde nicht durchgeführt")

runsTest(stream)

vnObs: 2097152
p-Wert: 0.9996104078054475
Die Sequenz hat den 'Runs Test' bestanden


# Test for the Longest Run of Ones in a Block (FIXME)

In [60]:

def longestRuns(stream):
    if len(stream) < 128:
        print("Not enough Data")
    elif len(stream) < 6272:
        m = 8
        k = 3
        n = 16
    elif len(stream) < 750000:
        m = 128
        k = 5
        n = 49
    else:
        m = 10**4
        k = 6
        n = 75

    vi = []
    anz = len(stream) // m
    start = 0
    end = m
    run = 0
    runs = []
    for i in range(anz):
        block = stream[start:end]
        for c in range(m):
            if block[c] == 1:
                run += 1
                one = True
            elif block[c] == 0 and one == True:
                one = False
                runs.append(run)
                run = 0
        runs.append(run)
        runs.sort()
        vi.append(runs[-1])





    

[3, 1, 2]
3


# Binary Matrix Rank Test

In [19]:
import numpy as np

def BinaryMatrixTest(stream, q):
    n = len(stream)//(q*q)
    start = 0
    end = q
    colums_list = []
    matrix = []
    count_max_rank = 0
    count_one_lower = 0
    count_else = 0
    for matrices in range(n):
        for row in range(0, q):
            for c in range(start, end):
                colums_list.append(stream[c])
            matrix.append(colums_list)
            colums_list = []
            start += q
            end += q
        matrix = np.matrix(matrix)
        rank = np.linalg.matrix_rank(matrix)
        if rank == q:
            count_max_rank = 1
        elif (rank + 1) == q:
            count_one_lower += 1
        matrix = []
    count_else = n - count_max_rank - count_one_lower
    chi_squared = (((count_max_rank - (0.2888*n))**2)/(0.2888*n)) + (((count_one_lower - (0.5776*n))**2)/(0.5776*n)) + (((count_else - (0.1336*n))**2)/(0.1336*n))
    pValue = math.e**((-1*chi_squared)/2)
    print(f"p-Wert: {pValue}")
    if pValue >= 0.01:
        print("Die Sequenz hat den 'Binary Matrix Rank Test' bestanden")
    else:
        print("Die Sequenz hat den 'Binary Matrix Rank Test' nicht bestanden")
    
BinaryMatrixTest(stream, 32)

p-Wert: 0.0
Die Sequenz hat den 'Binary Matrix Rank Test' nicht bestanden


# Vergleich mit Coinflip

In [27]:
import collections, numpy

coinflip_sequence = [0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0]

def analyse_Sequence(sequence):
    run_one = 0
    run_zero = 0
    runs_ones = []
    runs_zeroes = []
    counter_ones = 0
    for c in range(len(sequence)):
        if sequence[c] == 1:
            if run_zero != 0:
                runs_zeroes.append(run_zero)
            run_zero = 0
            run_one += 1
            counter_ones += 1
        elif sequence[c] == 0:
            if run_one != 0:
                runs_ones.append(run_one)
            run_one = 0
            run_zero += 1
    if run_one != 0:
        runs_ones.append(run_one)
    if run_zero != 0:
        runs_zeroes.append(run_zero)

    print(f"Runs of ones: {runs_ones}")
    print(f"Runs of zeroes: {runs_zeroes}")
    print(f"#Bits: {len(sequence)}")
    print(f"#Ones: {counter_ones}")
    print(f"#Zeroes: {len(sequence) - counter_ones}")
    runs_ones.sort()
    runs_zeroes.sort()
    print(f"Runs of ones (sorted by length): {runs_ones}")
    print(f"Runs of zeroes (sorted by length): {runs_zeroes}")
    counter = 0
    a = numpy.array(runs_ones)
    b = numpy.array(runs_zeroes)

    number_of_runs_per_length_ones = collections.Counter(a)
    number_of_runs_per_length_zeroes = collections.Counter(b)

    print(number_of_runs_per_length_ones)
    print(number_of_runs_per_length_zeroes)

print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("COINFLIP SEQUENCE:")
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
analyse_Sequence(coinflip_sequence)
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

state = 0b1000010010110001000011
length = 200

LFSR_sequence = lfsr(length, state)
print("LFSR SEQUENCE 1:")
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
analyse_Sequence(LFSR_sequence)
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

state = 0b1000010010110001000011
length = 30000


LFSR_sequence = lfsr(length, state)
LFSR_sequence = LFSR_sequence[25000:25200]

print("LFSR SEQUENCE 2:")
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
analyse_Sequence(LFSR_sequence)
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

LFSR_sequence = lfsr(length, state)
LFSR_sequence = LFSR_sequence[15000:15200]

print("LFSR SEQUENCE 3:")
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
analyse_Sequence(LFSR_sequence)
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")



-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
COINFLIP SEQUENCE:
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Runs of ones: [1, 1, 6, 3, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 3, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 3, 1, 4, 1, 3, 2, 4, 1, 2, 2, 6, 4, 3, 2, 2, 2, 4, 1, 1]
Runs of zeroes: [1, 1, 4, 1, 1, 1, 2, 2, 2, 1, 2, 2, 8, 2, 2, 1, 2, 3, 2, 1, 6, 1, 2, 1, 5, 1, 3, 2, 2, 3, 2, 1, 5, 1, 2, 3, 1, 1, 2, 3, 2, 1, 2, 3, 1, 2, 1, 1, 2, 1]
#Bits: 200
#Ones: 96
#Zeroes: 104
Runs of ones (sorted by length): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 6, 6]
Runs of zeroes (sorted by length): [1, 1, 1, 1, 1